## Tarea 5: utilizar la transformación por lotes para obtener inferencias a partir de un gran conjunto de datos

## Tarea 5.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from sagemaker.transformer import Transformer

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Guarde el modelo del laboratorio de entrenamiento y ajuste en el bucket predeterminado de Amazon Simple Storage Service (Amazon S3). Configure un modelo usando **create_model** y configure **ModelDataUrl** para referenciar al modelo entrenado.

In [ ]:
#set-up-model
# Upload the model and dataset to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

Cargue los registros de lotes en el bucket predeterminado de Amazon S3.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="batch_data.csv", Bucket=bucket, Key=f"{prefix}/batch_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/batch_data.csv"

### Tarea 5.2: crear un trabajo de transformación por lotes

La transformación por lotes administra automáticamente el procesamiento de grandes conjuntos de datos dentro de los límites de los parámetros especificados. Cuando se inicia un trabajo de transformación por lotes, SageMaker inicializa las instancias de cómputo y distribuye la inferencia o la carga de trabajo de preprocesamiento entre ellas. La transformación por lotes particiona los objetos de Amazon S3 de la entrada por claves y mapea dichos objetos a las instancias.

Use la transformación por lotes cuando necesite obtener inferencias de grandes conjuntos de datos o cuando no necesite un punto de enlace persistente.

Para crear un trabajo de transformación por lotes, debe configurar las siguientes opciones:
- **model_name**: el nombre de su modelo.
- **instance_type**: el tipo de instancia de Amazon Elastic Compute Cloud (Amazon EC2) que se debe usar; por ejemplo, "ml.c4.xlarge".
- **instance_count**: la cantidad de instancias de EC2 que se deben usar.
- **assemble_with**: la forma en que se ensambla la salida. Los valores válidos son “Line” (Línea) o “None” (Ninguno).
- **strategy**: la estrategia que se usó para decidir cómo reunir registros por lotes en una sola solicitud. Los valores válidos son "MultiRecord" y "SingleRecord".
- **accept**: el tipo de archivo que se debe aceptar.
- **output_path**: la ubicación en Amazon S3 para guardar el resultado de la transformación. Si no se especifica, los resultados se almacenan en un bucket predeterminado.

In [ ]:
#create-batch-transformer
transformer = Transformer(
    model_name=model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

Use el conjunto de datos de prueba como registros de sus clientes y ejecute el trabajo de transformación por lotes. La ejecución del trabajo puede demorar hasta 10 minutos con este conjunto de registros de clientes.

Consulte [Uso de la transformación por lotes](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) para obtener más información sobre los trabajos de transformación por lotes.

In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

### Tarea 5.3: ver los datos de predicción en Amazon S3

El trabajo de transformación por lotes almacena la salida en el bucket y la carpeta que especificó cuando configuró el transformador. Puede ver los resultados de la predicción en Amazon S3, ya sea en la Consola de administración de AWS o en el cuaderno.

Si quiere descargar y visualizar la salida de la consola, navegue a Amazon S3, abra el bucket que comienza con **sagemaker-** y vaya al objeto ubicado en **/sagemaker/mlasms/batch-transform/test**. Descargue el objeto **batch_data.csv.out** y ábralo con un editor de bloc de notas. El archivo contiene cientos de valores predichos para los registros de clientes que ejecutó a través del trabajo de transformación por lotes.

También se puede incluir en el cuaderno una muestra de la salida.


In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head batch_data.csv.out

### Conclusión

¡Felicitaciones! Usó Amazon SageMaker para ejecutar correctamente un trabajo de transformación por lotes.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.